In [1]:
%load_ext tensorboard

import tensorflow as tf
import pathlib
import os
import tensorflow_addons as tfa
import numpy
import datetime
from matplotlib import pyplot as plt
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, Reshape, UpSampling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
def preprocess_and_save():
    train_x, val_x = load_data()
    train_x_images = []
    val_x_images = []

    with tf.device('/cpu:0'):
        for x in train_x:
            image_string = tf.io.read_file(x)
            image = tf.image.decode_jpeg(image_string, channels = 3)
            image = tf.image.convert_image_dtype(image, tf.float32)
            image = tf.image.resize(image, [64, 64])
            image = numpy.array(image)
            train_x_images.append(image)

        for x in val_x:
            image_string = tf.io.read_file(x)
            image = tf.image.decode_jpeg(image_string, channels = 3)
            image = tf.image.convert_image_dtype(image, tf.float32)
            image = tf.image.resize(image, [64, 64])
            image = numpy.array(image)
            val_x_images.append(image)


    train_x_images = numpy.array(train_x_images)
    val_x_images = numpy.array(val_x_images)
    numpy.save(".//Data//Dog_numpy_train_64_color",train_x_images )
    numpy.save(".//Data//Dog_numpy_val_64_color", val_x_images)

In [3]:
def load_preprocessed():
    train_x_images = numpy.load(".//Data//Dog_numpy_train_64_color.npy")
    val_x_images = numpy.load(".//Data//Dog_numpy_val_64_color.npy")
    return train_x_images, val_x_images

In [4]:
def Encoder():
    input_img = Input(shape = (64, 64, 3))
    x = Conv2D(32, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(input_img)
    
    x = MaxPooling2D((2, 2), padding = 'same')(x)
    x = Conv2D(64, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)

    x = MaxPooling2D((2, 2), padding = 'same')(x)
    x = Conv2D(128, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)

    x = MaxPooling2D((2, 2), padding = 'same')(x)
    x = Flatten()(x)
    encoded = Dense(512)(x)

    return Model(input_img, encoded)

In [5]:
def Decoder():
    input_dense = Input(shape = (512))
    x = Dense(8192)(input_dense)
    x = Reshape((8,8,128))(x)
    x = Conv2D(128, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)    

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)  

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)

    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), 
                        activation='sigmoid', 
                        kernel_initializer='he_uniform', 
                        padding='same')(x)

    return Model(input_dense, decoded)


In [6]:
def AutoEncoder():
    input_img = Input(shape = (64, 64, 3))
    encoder = Encoder()
    encoded = encoder(input_img)
    decoder = Decoder()
    decoded = decoder(encoded)
    autoencoder = Model(input_img, decoded)
    autoencoder.summary()
    autoencoder.compile(optimizer='adam', 
                        loss='binary_crossentropy', 
                        metrics = ["accuracy"])

    return autoencoder

In [7]:
def Decoder():
    input_dense = Input(shape = (512))
    x = Dense(8192)(input_dense)
    x = Reshape((8,8,128))(x)
    x = Conv2D(128, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)    

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)  

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                padding='same')(x)

    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), 
                        activation='sigmoid', 
                        kernel_initializer='he_uniform', 
                        padding='same')(x)

    return Model(input_dense, decoded)




In [8]:
def AutoEncoder():
    input_img = Input(shape = (64, 64, 3))
    encoder = Encoder()
    encoded = encoder(input_img)
    decoder = Decoder()
    decoded = decoder(encoded)
    autoencoder = Model(input_img, decoded)
    autoencoder.summary()
    autoencoder.compile(optimizer='adam', 
                        loss='binary_crossentropy', 
                        metrics = ["accuracy"])

    return autoencoder


In [9]:
def train_model(train_images, val_images, model):
    os.system("!rm -rf ./logs/DOG_AUTO_64")
    # Train model while saving best validation
    filepath="weights_best_dog_auto_64.hdf5"
    checkpoint = ModelCheckpoint(filepath, 
                                    monitor='val_loss', 
                                    verbose=1, 
                                    save_best_only=True, 
                                    mode='min')

    log_dir = ".\\logs\\DOG_AUTO_64\\fit"
    tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                        histogram_freq=1, 
                                        profile_batch=0)

    es = EarlyStopping(monitor='val_loss', 
                        mode='min', 
                        verbose=1, 
                        patience=10)

    callbacks_list = [tb, es, checkpoint]

    model.fit(train_images, train_images,
                batch_size = 32,
                validation_data=(val_images, val_images), 
                epochs = 2000, 
                shuffle = True,
                callbacks = callbacks_list)

In [10]:
train, val = load_preprocessed()
ae = AutoEncoder()
train_model(train, val, ae)
 

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
functional_1 (Functional)    (None, 512)               4288064   
_________________________________________________________________
functional_3 (Functional)    (None, 64, 64, 3)         4443203   
Total params: 8,731,267
Trainable params: 8,731,267
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
274/274 [==============================] - ETA: 0s - loss: 0.6383 - accuracy: 0.5528
Epoch 00001: val_loss improved from inf to 0.60095, saving model to weights_best_dog_auto_64.hdf5
274/274 [==============================] - 5s 17ms/step - loss: 0.6383 - accuracy: 0.5528 - val_loss: 0.6009 - val_accuracy: 0.5612
Epoch 2/2000
273/274 [========================

InternalError: GPU sync failed

In [ ]:
test = ae.predict(val)

fig= plt.figure(figsize=(2,10))
col = 2
row = 10
count = 0
for i in range(1, col * row + 1, 2):
    if 1%2 ==1:
        img = val[count]
        fig.add_subplot(row, col, i)
        plt.imshow(img)
        img = test[count]
        fig.add_subplot(row, col, i + 1)
        plt.imshow(img) 
        count += 1
plt.show()
        